Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. It is to download the data onto the machine's storage.
To replicate the same on a local machine use the link to download the dataset and seperate it into the respective folders.

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip \
    -O /tmp/happy-or-sad.zip

--2020-06-26 14:44:32--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-06-26 14:44:32 (108 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data.
To replicate on local system mention the local path to the images.

In [4]:
import os
import tensorflow as tf
import zipfile

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [9]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('acc') > DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

    callbacks = myCallback()

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')      
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
                 optimizer=RMSprop(lr=0.001),
                 metrics=['acc'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    history = model.fit_generator(
        train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

    return history.history['acc'][-1]

In [10]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
2/2 [==============================] - 0s 114ms/step - loss: 5.6042 - acc: 0.4000
Epoch 2/15
2/2 [==============================] - 0s 110ms/step - loss: 0.9784 - acc: 0.6500
Epoch 3/15
2/2 [==============================] - 0s 115ms/step - loss: 0.6986 - acc: 0.4000
Epoch 4/15
2/2 [==============================] - 0s 111ms/step - loss: 0.7833 - acc: 0.5000
Epoch 5/15
2/2 [==============================] - 0s 109ms/step - loss: 0.6007 - acc: 0.7500
Epoch 6/15
2/2 [==============================] - 0s 112ms/step - loss: 0.6048 - acc: 0.6000
Epoch 7/15
2/2 [==============================] - 0s 112ms/step - loss: 0.3742 - acc: 0.9500
Epoch 8/15
2/2 [==============================] - ETA: 0s - loss: 0.1724 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
2/2 [==============================] - 0s 116ms/step - loss: 0.1724 - acc: 1.0000


1.0